In [3]:
import pandas as pd
import numpy as np
import os

In [80]:
train_data_folder = r'UCI HAR Dataset\train\Inertial Signals'
sensor_names = [filename.replace('_train.txt','').replace('_test.txt','') for filename in os.listdir(train_data_folder)]
test_data_folder = r'UCI HAR Dataset\test\Inertial Signals'

In [81]:
sensor_names

['body_acc_x',
 'body_acc_y',
 'body_acc_z',
 'body_gyro_x',
 'body_gyro_y',
 'body_gyro_z',
 'total_acc_x',
 'total_acc_y',
 'total_acc_z']

In [125]:
train_files = [os.path.join(train_data_folder, filename + '_train.txt') for filename in sensor_names]
test_files = [os.path.join(test_data_folder, filename + '_test.txt') for filename in sensor_names]

def read_data(files):
    data_matrix = []
    for filename in files:
        single_sensor_matrix = pd.read_csv(filename, delim_whitespace=True, header=None).values
        data_matrix.append(single_sensor_matrix)
    data_matrix = np.array(data_matrix)
    data_matrix = np.transpose(data_matrix,(1,2,0))
    return data_matrix

X_train = read_data(train_files)
X_test = read_data(test_files)

print('Train data shape: ',X_train.shape)
print('Test data shape: ',X_test.shape)

- There are 6 accelerometer data and 3 gyroscope (9 sensors) data with 128 features and 7352 time signals. 

In [ ]:
# Each row is a 2.56 second long window of data
print('Total duration in train data is around', int(7352*2.56/3600), 'hours of activities')

Total duration in train data is around 5 hours of activities


In [ ]:
def get_labels(split):
    df = pd.read_csv(f'UCI HAR Dataset\\{split}\y_{split}.txt', delim_whitespace=True, header=None)
    return pd.get_dummies(df[0])
y_train = get_labels('train')
y_test = get_labels('test')
y_train.head()

,1,2,3,4,5,6
0,0,0,0,0,1,0
1,0,0,0,0,1,0
2,0,0,0,0,1,0
3,0,0,0,0,1,0
4,0,0,0,0,1,0


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout

In [123]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32
n_classes = y_train.shape[1]
input_dim = train_data.shape[2]
timesteps = train_data.shape[1]
n_samples = train_data.shape[0]

# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 32)                5376      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 6)                 198       
                                                                 
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [124]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)